In [1]:
# Installing packages
from IPython.display import clear_output 
! pip install spektral
! pip install rdkit
! pip install ogb
! pip install py3Dmol

clear_output()

In [ ]:
# Aebel's paths
from google.colab import drive
drive.mount("/content/drive")
path = "/content/drive/My Drive/University/MPHYS_Shared_folder/Our_Peptide_GCN/Data/AllData.csv"
model_path =  "/content/drive/My Drive/University/MPHYS_Shared_folder/Our_Peptide_GCN/Data/model_runs/v run 3/v run 3_model.h5"
dataset_path = "/content/drive/My Drive/University/MPHYS_Shared_folder/Our_Peptide_GCN/Data/AMP_Dataset.pickle"

Mounted at /content/drive


In [18]:
# Jussi's path
from google.colab import drive
drive.mount("/content/drive")
path = "/content/drive/My Drive/University/AllData.csv"
model_path =  "/content/drive/My Drive/University/model.h5"
dataset_path = "/content/drive/My Drive/University/AMP_Dataset.pickle"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Dataset

In [19]:
from spektral.utils import pad_jagged_array
from spektral.data.utils import sp_matrices_to_sp_tensors
from spektral.data.utils import batch_generator
import numpy as np

def graph_to_tuple(graph):
    """Converts a graph object into a tuple. 
    Copied from spektral source code.

    Args:
        graph (Spektral.data.graph.Graph)
    
    Returns:
        tuple (tuple): The input graph but as a tuple
    """
    return tuple([graph.x, graph.a, graph.e, graph.metadata, graph.y])

def pack(batch):
    """Converts graph objects to dictionary of lists of each argument.
    Copied from spektral source code.

    Args:
        batch (list): List of graphs objects.
    
    Returns:
        dict (dict): Dictionary which contain node features, edge features, 
          adjacency matrix, metadata vector and labels
    """
    output = [list(elem) for elem in zip(*[graph_to_tuple(g) for g in batch])]
    keys = ["x_list", "a_list", "e_list", "metadata", "y_list" ]
    return dict(zip(keys, output))

def to_batch(x_list=None, a_list=None, e_list=None, metadata=None, mask=False):
    """Pads list of arrays such that they have then same size for each batch.
    Copied from spektral source code.

    Args:
        x_list (list)
        a_list (list)
        e_list (list)
        metadata (list)
        mask (boolean)
    """
    if a_list is None and x_list is None:
        raise ValueError("Need at least x_list or a_list")
    
    # Max number of nodes in the node features list
    n_max = max([x.shape[0] for x in (x_list if x_list is not None else a_list)])

    # Node features
    x_out = None
    if x_list is not None:
        x_out = pad_jagged_array(x_list, (n_max, -1))

    # Adjacency matrix
    a_out = None
    if a_list is not None:
        if hasattr(a_list[0], "toarray"):  # Convert sparse to dense
            a_list = [a.toarray() for a in a_list]
        a_out = pad_jagged_array(a_list, (n_max, n_max))

    # Edge attributes
    e_out = None
    if e_list is not None:
        if e_list[0].ndim == 2:  # Sparse to dense
            for i in range(len(a_list)):
                a, e = a_list[i], e_list[i]
                e_new = np.zeros(a.shape + e.shape[-1:])
                e_new[np.nonzero(a)] = e
                e_list[i] = e_new
        e_out = pad_jagged_array(e_list, (n_max, n_max, -1))

    # Metadata
    metadata_out = tf.convert_to_tensor([metadata], dtype=tf.float32)
    return tuple(out for out in [x_out, a_out, e_out, metadata] if out is not None)

def collate(batch):
    '''This takes as input a list of Graph objects and returns a list of 
    Tensors.

    Args:
        batch (list): List of graph objects.
    Returns:
        output (tuple): Node features, edge features, adjacency matrix, metadata
          vector.
        y (int): Label.
    '''
    packed = pack(batch)
    y = packed.pop("y_list", None)
    if y is not None:
        y = np.array(y)
    output = to_batch(**packed, mask=False)
    output = sp_matrices_to_sp_tensors(output)
    x, a,e, m = output
    m = tf.convert_to_tensor(m, dtype=tf.float32)
    output = x, a, e, m
    if len(output) == 1:
        output = output[0]
    if y is None:
        return output
    else:
        return output, y

class Data_Loader:
    '''
    Converts the numpy array of graph objects to a generator object which can be
    passed into the keras model. 
    Works like:  https://graphneural.network/loaders/#batchloader but adjusted 
    so we could feed in the metadata for each peptide into the model.
    '''
    def __init__(self, dataset, batch_size=1, epochs=None):
        """
        """
        self.dataset = dataset
        self.batch_size = batch_size
        self.epochs = epochs
        self._generator = self.generator()
        self.steps_per_epoch = int(np.ceil(len(self.dataset) / self.batch_size))

    def __iter__(self):
        return self

    def __next__(self):
        """
        """
        nxt = self._generator.__next__()
        return collate(nxt)

    def generator(self):
        """
        """
        return batch_generator(
            self.dataset,
            batch_size=self.batch_size,
            epochs=self.epochs,
            shuffle=False,
            )

    def load(self):
      return self

#beep boop

In [20]:
# Loading the AMP_dataset from a file (make sure you created the dataset before hand in the model)
import pickle
with open(dataset_path, "rb") as f:
  amp_data = pickle.load(f)

AttributeError: ignored

In [5]:
# Loading saved model
import numpy, random
from tabulate import tabulate
import tensorflow as tf
import spektral
from spektral.data import Dataset
from spektral.data import DisjointLoader
from spektral.layers import ECCConv, GlobalSumPool

dataset =  amp_data
model = tf.keras.models.load_model(model_path, custom_objects={'ECCConv': ECCConv, "GlobalSumPool" : GlobalSumPool})

NameError: ignored

In [8]:
from ipywidgets.widgets.widget_color import ColorPicker
from ipywidgets import interact,fixed,IntSlider, VBox, Text
import ipywidgets
from rdkit import Chem

import py3Dmol
from IPython.display import display

def predict_graph(graph_index, dataset):
  test_graph = Data_Loader(dataset[[graph_index]], batch_size=1, epochs=1)
  prediction = model.predict(test_graph, verbose=0)[0][0]
  actual_value = dataset[graph_index].y
  return prediction, actual_value[0]

def MolTo3DView(mol, size=(500, 300), style="stick", surface=True, opacity=0.5):
    """Draw molecule in 3D
    
    Args:
    ----
        mol: rdMol, molecule to show
        size: tuple(int, int), canvas size
        style: str, type of drawing molecule
               style can be 'line', 'stick', 'sphere', 'carton'
        surface, bool, display SAS
        opacity, float, opacity of surface, range 0.0-1.0
    Return:
    ----
        viewer: py3Dmol.view, a class for constructing embedded 3Dmol.js views in ipython notebooks.
    """
    assert style in ('line', 'stick', 'sphere', 'carton')
    mblock = Chem.MolToMolBlock(mol)
    viewer = py3Dmol.view(width=size[0], height=size[1])
    viewer.addModel(mblock, 'mol')
    viewer.setStyle({style:{}})
    if surface:
        viewer.addSurface(py3Dmol.SAS, {'opacity': opacity})
    viewer.zoomTo()
    return viewer

def conf_viewer(idx):
    mol = Chem.MolFromFASTA(amp_data[idx].fasta)
    return MolTo3DView(mol).show()

def predict_graph_viewer(idx):
  prediction, actual_value = predict_graph(idx, dataset)
  prediction_text = "Prediction: " + f'{float(f"{prediction:.3g}"):g}' 
  actual_value_text = "Actual Value: " + str(actual_value)
  box = VBox( [ipywidgets.Label(value=prediction_text), ipywidgets.Label(value=actual_value_text) ] )
  return display(box)



index = ipywidgets.BoundedIntText(
    value=0,
    min=0,
    max=len(dataset)-1,
    step=1,
    description='Peptide Idx:',
    disabled=False
    )

def choose_random_peptide(button):
  index.value = random.randint(0, len(dataset)-1)



molecule_3d_view = ipywidgets.interactive_output(conf_viewer, {'idx': index})
prediction_label = ipywidgets.interactive_output(predict_graph_viewer,  {'idx': index})
randomize_button = ipywidgets.Button(
    description='Random Peptide',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='pick a random peptide',
)
randomize_button.on_click(choose_random_peptide)

display(VBox([index, randomize_button]))
display(molecule_3d_view)
display(prediction_label)


NameError: ignored

# Prediction on new peptides

In [16]:
from ogb.utils.features import (atom_to_feature_vector, bond_to_feature_vector)
from scipy.sparse import csr_matrix
from rdkit import Chem
import numpy as np
from ipywidgets.widgets.widget_color import ColorPicker
from ipywidgets import interact,fixed,IntSlider, VBox, Text
import ipywidgets

import py3Dmol
from IPython.display import display

def smiles_to_graph(smiles_string):
    """
    Converts SMILES string to graph Data object
    :input: SMILES string (str)
    :return: graph object
    """

    mol = Chem.MolFromFASTA(smiles_string)
    if mol == None:
        print(smiles_string)
        print('Invalid smile')
        breakpoint()

    # atoms
    atom_features_list = []
    for atom in mol.GetAtoms():
        ftrs = atom_to_feature_vector(atom)
        atom_features_list.append(ftrs)
    node_features = np.array(atom_features_list, dtype = np.float32)

    # bonds
    num_bond_features = 3  # bond type, bond stereo, is_conjugated
    if len(mol.GetBonds()) > 0: # mol has bonds
        edge_features_list = []
        edges_list = []
        for bond in mol.GetBonds():
            i = bond.GetBeginAtomIdx()
            j = bond.GetEndAtomIdx()
            edge_feature = bond_to_feature_vector(bond)

            # add edges in both directions      
            edges_list.append((i,j))
            edge_features_list.append(edge_feature)
            edges_list.append((j,i))
            edge_features_list.append(edge_feature)

        # data.edge_index: Sparse adajacency matrix with shape [n_nodes, n_nodes]
        edge_index_np = np.array(edges_list, dtype = np.float32)

        # data.edge_attr: Edge feature matrix with shape [num_edges, num_edge_features]
        edge_attr = np.array(edge_features_list, dtype = np.float32)
        
        edge_index_np, edge_attr = spektral.utils.reorder(
            edge_index_np, edge_features=edge_attr)
        edge_index = csr_matrix(
            ([1.0 for num in range(len(edge_index_np))], 
             (edge_index_np[:,0], edge_index_np[:,1])),
             shape=(len(node_features), len(node_features))
             , dtype=np.float64
             )

    return edge_attr, edge_index, node_features

class empty_dataset(Dataset):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
    
    def read(self):
      return [spektral.data.graph.Graph()]

a = empty_dataset()
a[0] = spektral.data.graph.Graph()

def predict_on_new_peptide(fasta_sequence):
  y = 0.5
  e, a, x = smiles_to_graph(fasta_sequence)
  new_graph = spektral.data.graph.Graph(x, a, e, y, fasta=fasta_sequence, metadata= [0,0,0,0,0,0,0,0])
  a = empty_dataset()
  a[0] = new_graph
  loaded_graph = Data_Loader(a, batch_size=1, epochs=1)
  prediction = model.predict(loaded_graph, verbose=0)[0][0]
  return prediction

def conf_viewer(fasta):
  mol = Chem.MolFromFASTA(fasta)
  return MolTo3DView(mol).show()

def predict_graph_label(fasta):
  prediction = predict_on_new_peptide(fasta)
  prediction_text = "Prediction: " + f'{float(f"{prediction:.3g}"):g}' 
  box = ipywidgets.Label(value=prediction_text)
  return display(box)

fasta_input = ipywidgets.Text(
    value=amp_data[0].fasta,
    placeholder='Type something',
    description='String:',
    disabled=False
)



molecule_3d_view = ipywidgets.interactive_output(conf_viewer, {'fasta': fasta_input} )
prediction_label = ipywidgets.interactive_output(predict_graph_label,  {'fasta': fasta_input})

display(fasta_input)
display(molecule_3d_view)
display(prediction_label)

/usr/local/lib/python3.9/dist-packages/spektral/data/dataset.py:120: UserWarning: The graphs in this dataset have no adjacency matrix. Is this intentional?
  warnings.warn(


NameError: ignored

Observations:
> bigger the peptide the more likely it is to be amp??
